# Pandas 🐼🐼🐼

### Miglior strumento di data wrangling

Pandas è ideale per il data wrangling, ovvero il processo di trasformazione e pulizia dei dati grezzi per analisi e modelli, mentre NumPy/array eccelle nei calcoli intensivi una volta che il dataset è pronto.

| Esigenza tipica                                                  | Funzionalità Pandas                     | Quando serve                                                                |
| ---------------------------------------------------------------- | --------------------------------------- | --------------------------------------------------------------------------- |
| 1. **Importare** formati eterogenei (CSV, Excel, TSV, JSON, SQL) | `read_*()`                              | Ogni volta che i dati provengono da laboratori, EHR, registry o altri dispositivi |
| 2. **Pulire** dati sporchi o parziali                            | `dropna`, `fillna`, `astype`, `replace` | Prima di qualsiasi statistica o di usarli per Machine Learning                                          |
| 3. **Ristrutturare** il dataset (modificarne la struttura per facilitarne l’analisi o per adattarlo a specifiche esigenze)                                  | `melt`, `pivot`, `stack/unstack`        | Per allineare o sistemare misure longitudinali, matrici di espressione...                 |
| 4. **Filtrare & creare sottogruppi**                                    | Boolean indexing, `query`, `groupby`    | Focus su un fenotipo, un trattamento, un gene...                               |
| 5. **Aggregare & statistiche rapide**                            | `groupby().agg()`, `describe`           | Report clinici, cruscotti QC (Quality Control)...                                                |
| 6. **Merge/Join** di più fonti                                   | `merge`, `concat`                       | Integrare imaging + clinica + omics (Dati omici → Genomica, trascrittomica, proteomica, metabolomica)                                         |
| 7. **Time series per il monitoraggio continuo**                                               | `to_datetime`, `resample`, `rolling`    | Wearable, follow-up, Analisi di segnali continui in terapia intensiva (ICU waveform analysis)...                                           |


### Perché non basta usare liste Python o array NumPy al posto di Pandas?

| Esigenza tipica nei dati biomedici                     | Liste Python                                | NumPy array                                           | Pandas DataFrame                                |
| ------------------------------------------------------ | ------------------------------------------- | ----------------------------------------------------- | ----------------------------------------------- |
| **Etichette (ID paziente, nome gene, tempo)**          | ❌ gestite “a mano” con strutture ausiliarie | ❌ non esistono; solo indici interi                    | ✅ `index`, `columns`, gerarchici (`MultiIndex`) |
| **Gestire dati eterogenei** (numeri + stringhe + date)         | ✅ ma zero funzioni vettoriali               | ⚠️ possibile con `dtype=object`, ma perde in performance | ✅ colonne con tipi diversi, ottimizzate         |
| **Valori mancanti** (`NaN`, `None`, sentinel)          | ❌ serve logica ad-hoc                       | ⚠️ `np.nan` solo per float; complicato per tipi misti | ✅ `isna` per identificare valori mancanti, `fillna`per riempire valori mancanti, propagazione coerente per propagare il valore precedente o successivo nelle celle vuote.       |
| **Operazioni “SQL-like”** (`groupby`, `join`, `pivot`) | ❌ codice manuale e ciclo esplicito          | ❌ non previste                                        | ✅ una riga di codice idiomatica (ovvero con funzioni intuitive e efficienti)                |
| **Time series**             | ❌ da implementare                           | ⚠️ serve `np.searchsorted`/for loop                   | ✅ API dedicate (`resample` per aggregazione a intervalli temporali, `rolling` per applicare operazioni statistiche su una finestra mobile di dati, `shift` per confrontare valori tra istanti di tempo diversi) |
| **I/O (input/output) nativo** (CSV, Excel, SQL, Parquet)              | ❌ librerie esterne, parsing manuale         | ❌ idem                                                | ✅ `read_*`, `to_*` tutte funzioni one-liner                    |
| **Leggibilità notebook/paper**                         | 😐 spesso 3-4 strutture parallele           | 😐 array senza intestazioni esplicite                             | 😀 tabelle auto-formattate                      |


### 🏥 1 | Analisi di dataset clinici multifattoriali
(= studiare dati sanitari considerando più variabili contemporaneamente, come età, sesso, genetica, stile di vita, parametri fisiologici ...)

Scenario:
Analisi di parametri clinici di pazienti con diverse diagnosi per identificare correlazioni e pattern significativi.

In [13]:
!pip install pandas
import pandas as pd

# Caricamento dati clinici di esempio
df_clinici = pd.DataFrame({
    'paziente_id': range(1, 11),
    'età': [45, 52, 67, 38, 71, 49, 55, 62, 41, 58],
    'genere': ['M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F'],
    'glicemia': [95, 105, 142, 88, 170, 98, 110, 130, 92, 115],
    'pressione_sistolica': [120, 135, 155, 110, 165, 125, 140, 150, 118, 145],
    'colesterolo': [180, 220, 250, 175, 270, 190, 230, 245, 185, 235],
    'diagnosi': ['sano', 'pre-diabete', 'diabete', 'sano', 'diabete', 
                'sano', 'pre-diabete', 'diabete', 'sano', 'pre-diabete']
})

# Statistiche descrittive per gruppo diagnostico
stats_per_diagnosi = df_clinici.groupby('diagnosi').agg({
    'età': ['mean', 'std'],
    'glicemia': ['mean', 'std'],
    'pressione_sistolica': ['mean', 'std'],
    'colesterolo': ['mean', 'std']
})

print("Statistiche per gruppo diagnostico:")
print(stats_per_diagnosi)

# Correlazione tra parametri
corr = df_clinici[['età', 'glicemia', 'pressione_sistolica', 'colesterolo']].corr()
print("\nMatrice di correlazione:")
print(corr)

# Identifica pazienti con glicemia alta (>125)
pazienti_glicemia_alta = df_clinici[df_clinici['glicemia'] > 125]
print("\nPazienti con glicemia alta:")
print(pazienti_glicemia_alta[['paziente_id', 'età', 'genere', 'glicemia', 'diagnosi']])


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Statistiche per gruppo diagnostico:
                   età              glicemia            pressione_sistolica  \
                  mean       std        mean        std                mean   
diagnosi                                                                      
diabete      66.666667  4.509250  147.333333  20.526406          156.666667   
pre-diabete  55.000000  3.000000  110.000000   5.000000          140.000000   
sano         43.250000  4.787136   93.250000   4.272002          118.250000   

                      colesterolo             
                  std        mean        std  
diagnosi                                      
diabete      7.637626  255.000000  13.228757  
pre-diabete  5.000000  228.333333   7.637626  
sano         6.238322  182.500000   6.454972  

Matrice di correlazione:
                          età  glicemia  pressione_sistolica  colest

Utilità:
- Pulizia nomenclatura diagnosi
- Statistiche descrittive per reparto
- Individuazione outlier (df[df["HbA1c"]>12])
- Esportazione rapida in Excel per il board clinico (stats.to_excel("summary.xlsx"))

{Tabella heat-map dei valori medi di HbA1c per diagnosi}

## Appunto sui file
###Cosa sono

Pandas eccelle nella gestione di file di dati, che è uno dei suoi punti di forza principali. Questo codice include:
- Lettura e scrittura di diversi formati file: CSV, Excel, TSV, JSON, SQL
- Opzioni di lettura avanzate: gestione separatori, encoding, intestazioni personalizzate, - - gestione dei valori mancanti

Un file è un contenitore di dati salvati su disco (es. CSV, TXT, Excel, immagini, audio...)
Python può aprire, leggere, modificare e salvare file, come se li “sfogliasse riga per riga” o li caricassi in memoria come tabelle.
.

### Modi principali per lavorare con file in Python
1.File di testo (.txt, .csv, .tsv, .json, .log)

In [14]:
with open("dati.txt", "r") as f:
    contenuto = f.read()         # Legge tutto il file come stringa

FileNotFoundError: [Errno 2] No such file or directory: 'dati.txt'

2. File strutturati come tabelle (CSV, Excel, TSV)
Qui entra in gioco Pandas, che ti fa risparmiare ore:

In [ ]:
import pandas as pd

df = pd.read_csv("pazienti.csv")         # CSV = valori separati da virgole
df = pd.read_excel("analisi.xlsx")       # Legge un file Excel
df = pd.read_json("dati.json")           # Legge un file JSON

Pandas riconosce automaticamente il formato dei dati, crea una tabella (DataFrame) e assegna intestazioni e indici.

### Come si integrano i file con le librerie scientifiche
| Obiettivo                | Funzione Python/Pandas            |
| ------------------------ | --------------------------------- |
| Leggere un file          | `open()`, `pd.read_csv()`, ecc.   |
| Scrivere su file         | `write()`, `df.to_csv()`          |
| Elaborare i dati         | Manipolazione tramite Pandas      |
| Lavorare con file grandi | `chunksize`, `Parquet`, `feather` |

Esempio:

In [ ]:
df = pd.read_csv("espressione_genica.tsv", sep="\t")  # leggo da file
df.to_csv("risultati_filtrati.csv")                   # salvo su file

### 🧬 2 | Trascrittomica / RNA-seq
Scenario:
Analisi di dati di espressione genica per identificare geni differenzialmente espressi tra campioni di controllo e trattati.

In [15]:
import math
import pandas as pd

# --- 1. Caricamento -----------------------------------------------------------
counts = pd.read_csv("counts_matrix.tsv", sep="\t", index_col=0)   # righe = gene_id
meta   = pd.read_csv("samples.tsv",       sep="\t")                # sample_id, condition

# --- 2. Selezione campioni ----------------------------------------------------
ctrl_cols  = meta.loc[meta["condition"] == "control",  "sample_id"]
treat_cols = meta.loc[meta["condition"] == "treated",  "sample_id"]

# --- 3. Normalizzazione CPM (counts-per-million) ------------------------------
cpm = counts.div(counts.sum(axis=0), axis=1) * 1_000_000   # resta DataFrame

# --- 4. Medie di espressione per condizione -----------------------------------
mean_ctrl  = cpm[ctrl_cols].mean(axis=1)
mean_treat = cpm[treat_cols].mean(axis=1)

# --- 5. Log2 fold-change (solo Pandas + math) ---------------------------------
log2fc = ((mean_treat + 1) / (mean_ctrl + 1)).apply(math.log2)

# --- 6. DataFrame finale ------------------------------------------------------
deg = pd.concat(
    [mean_ctrl.round(2).rename("mean_ctrl"),
     mean_treat.round(2).rename("mean_treat"),
     log2fc.round(3).rename("log2FC")],
    axis=1
)

# --- 7. Filtro preliminare sui candidati --------------------------------------
candidati = deg[deg["log2FC"].abs() > 1]

# --- 8. Esportazione ----------------------------------------------------------
candidati.to_csv("DEG_pre_screen.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'counts_matrix.tsv'

Utilità:
- Lettura in Parquet → RAM-efficient
- Filtri basati su media/varianza
- Calcolo di reti di co-espressione
- Output diretto verso Seaborn/Scanpy

### 🧠 3 | Imaging e Quantificazione
Scenario:
Quantificazione di parametri morfometrici cellulari in risposta a diversi trattamenti farmacologici.

In [16]:
import pandas as pd

# Simuliamo dati di morfologia cellulare
cell_data = pd.DataFrame({
    'cell_id': range(1, 21),
    'trattamento': ['controllo']*7 + ['farmaco_A']*7 + ['farmaco_B']*6,
    'area': [100, 95, 105, 98, 102, 97, 103, 
             82, 78, 85, 80, 83, 77, 81, 
             125, 130, 128, 135, 122, 132],
    'perimetro': [35, 33, 36, 34, 35, 32, 36,
                  30, 28, 31, 29, 30, 27, 30,
                  40, 42, 41, 44, 39, 43],
    'intensità_nucleo': [120, 115, 125, 118, 122, 116, 124,
                        150, 145, 155, 148, 152, 142, 154,
                        100, 95, 105, 90, 110, 98]
})

# Statistiche per gruppo di trattamento
stats_morfologia = cell_data.groupby('trattamento').agg({
    'area': ['count', 'mean', 'std', 'min', 'max'],
    'perimetro': ['mean', 'std'],
    'intensità_nucleo': ['mean', 'std']
})

print("Statistiche morfometriche per trattamento:")
print(stats_morfologia)

# Calcolo di feature derivate
cell_data['ratio_area_perimetro'] = cell_data['area'] / cell_data['perimetro']

# Statistiche della feature derivata
ratio_stats = cell_data.groupby('trattamento')['ratio_area_perimetro'].agg(['mean', 'std'])
print("\nRapporto area/perimetro per trattamento:")
print(ratio_stats)

# Identificazione delle cellule con area molto grande
threshold = cell_data['area'].mean() + cell_data['area'].std()
cellule_grandi = cell_data[cell_data['area'] > threshold]
print(f"\nCellule con area superiore alla media + deviazione standard ({threshold:.2f}):")
print(cellule_grandi[['cell_id', 'trattamento', 'area']])


Statistiche morfometriche per trattamento:
             area                                  perimetro            \
            count        mean       std  min  max       mean       std   
trattamento                                                              
controllo       7  100.000000  3.559026   95  105  34.428571  1.511858   
farmaco_A       7   80.857143  2.794553   77   85  29.285714  1.380131   
farmaco_B       6  128.666667  4.718757  122  135  41.500000  1.870829   

            intensità_nucleo            
                        mean       std  
trattamento                             
controllo         120.000000  3.872983  
farmaco_A         149.428571  4.755949  
farmaco_B          99.666667  7.118052  

Rapporto area/perimetro per trattamento:
                 mean       std
trattamento                    
controllo    2.905942  0.059961
farmaco_A    2.762589  0.047845
farmaco_B    3.101391  0.027712

Cellule con area superiore alla media + deviazione standard (12

Utilità:
- Filtraggio rapido su soglia volumetrica
- Calcolo delta-volume fra follow-up
- Facilita il merge con outcome clinici

### 🧫 4 | High-Content Screening (96-well, 384-well)
Scenario:
Screening di composti in piastra multi-well per identificare molecole attive attraverso saggi di viabilità cellulare.

In [8]:
import pandas as pd

# Simulazione di dati di high-content screening (piastra 96-well)
# Definizione della piastra (semplificata, solo 24 pozzetti)
rows = list('ABCD')
cols = list(range(1, 7))
wells = [f"{r}{c}" for r in rows for c in cols]

# Creazione dei dati
hcs_data = pd.DataFrame({
    'well': wells,
    'compound': ['DMSO']*4 + ['Positivo']*4 + 
                ['Comp_1']*4 + ['Comp_2']*4 + ['Comp_3']*4 + ['Comp_4']*4,
    'concentration_uM': [0]*4 + [10]*4 + [1, 3, 10, 30]*4,
    'viability_pct': [
        # DMSO (controllo negativo)
        98, 102, 97, 103,
        # Controllo positivo
        25, 22, 18, 20,
        # Comp_1
        95, 85, 70, 30,
        # Comp_2
        100, 98, 95, 92,
        # Comp_3
        90, 60, 40, 20,
        # Comp_4
        85, 80, 50, 35
    ]
})

print("Dati screening:")
print(hcs_data)

# Calcolo media dei controlli
mean_neg_ctrl = hcs_data[hcs_data['compound'] == 'DMSO']['viability_pct'].mean()
mean_pos_ctrl = hcs_data[hcs_data['compound'] == 'Positivo']['viability_pct'].mean()

print(f"\nMedia controllo negativo (DMSO): {mean_neg_ctrl:.2f}%")
print(f"Media controllo positivo: {mean_pos_ctrl:.2f}%")

# Normalizzazione rispetto ai controlli (0% = controllo positivo, 100% = DMSO)
hcs_data['normalized_response'] = 100 * (hcs_data['viability_pct'] - mean_pos_ctrl) / (mean_neg_ctrl - mean_pos_ctrl)

# Pivot table per visualizzare l'effetto dose-risposta
dose_response = hcs_data[~hcs_data['compound'].isin(['DMSO', 'Positivo'])].pivot_table(
    index='compound', 
    columns='concentration_uM', 
    values='normalized_response',
    aggfunc='mean'
)

print("\nEffetto dose-risposta (% normalizzata):")
print(dose_response)

# Identificazione dei composti attivi (hit) a 10 µM
hits_10uM = hcs_data[(hcs_data['concentration_uM'] == 10) & 
                    (hcs_data['compound'] != 'DMSO') & 
                    (hcs_data['compound'] != 'Positivo') & 
                    (hcs_data['normalized_response'] < 50)]

print("\nComposti attivi (hit) a 10 µM (< 50% viabilità normalizzata):")
print(hits_10uM[['compound', 'normalized_response']])

Dati screening:
   well  compound  concentration_uM  viability_pct
0    A1      DMSO                 0             98
1    A2      DMSO                 0            102
2    A3      DMSO                 0             97
3    A4      DMSO                 0            103
4    A5  Positivo                10             25
5    A6  Positivo                10             22
6    B1  Positivo                10             18
7    B2  Positivo                10             20
8    B3    Comp_1                 1             95
9    B4    Comp_1                 3             85
10   B5    Comp_1                10             70
11   B6    Comp_1                30             30
12   C1    Comp_2                 1            100
13   C2    Comp_2                 3             98
14   C3    Comp_2                10             95
15   C4    Comp_2                30             92
16   C5    Comp_3                 1             90
17   C6    Comp_3                 3             60
18   D1    Comp

Utilità
- pivot_table rende il dataset “wide” per calcolo Δ
- Rank dei top-hits per successiva validazione
- Integrare annotazioni chimiche con merge

{Foto di piastra 384-well con pozzi colorimetrici + barplot crescita}

### 💉 5. Studi di Coorte Longitudinali
Scenario:
Analisi longitudinale di pazienti in uno studio clinico per valutare l'efficacia di diversi trattamenti nel tempo.

In [9]:
import pandas as pd

# Creazione di un dataset longitudinale semplificato
# Dati baseline (visita iniziale)
baseline_data = pd.DataFrame({
    'patient_id': range(1, 11),
    'age': [45, 52, 67, 38, 71, 49, 55, 62, 41, 58],
    'gender': ['M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F'],
    'treatment': ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B'],
    'baseline_score': [50, 55, 60, 45, 65, 52, 58, 63, 48, 56]
})

# Creazione dati di follow-up
followup_data = []

# Valori dei punteggi alle visite di follow-up
# Paziente 1-5 (trattamento A)
month3_scores_A = [45, 48, 52, 40, 58]
month6_scores_A = [40, 42, 45, 38, 52]
month12_scores_A = [35, 36, 40, 35, 48]

# Paziente 6-10 (trattamento B)
month3_scores_B = [50, 55, 60, 46, 54]
month6_scores_B = [48, 52, 58, 45, 52]
month12_scores_B = [47, 50, 55, 44, 50]

# Costruzione dei dati di follow-up
for visit in [3, 6, 12]:
    for i in range(10):
        patient_id = i + 1
        
        # Simula dropout (paziente 5 esce dallo studio dopo 3 mesi)
        if patient_id == 5 and visit > 3:
            continue
            
        # Assegna il punteggio in base al gruppo e alla visita
        if patient_id <= 5:  # Gruppo A
            if visit == 3:
                score = month3_scores_A[patient_id-1]
            elif visit == 6:
                score = month6_scores_A[patient_id-1]
            else:  # visit == 12
                score = month12_scores_A[patient_id-1]
        else:  # Gruppo B
            if visit == 3:
                score = month3_scores_B[patient_id-6]
            elif visit == 6:
                score = month6_scores_B[patient_id-6]
            else:  # visit == 12
                score = month12_scores_B[patient_id-6]
                
        followup_data.append({
            'patient_id': patient_id,
            'visit_month': visit,
            'score': score
        })

# Creazione DataFrame follow-up
followup_df = pd.DataFrame(followup_data)

# Unione con i dati baseline
long_data = pd.merge(
    followup_df,
    baseline_data[['patient_id', 'age', 'gender', 'treatment', 'baseline_score']],
    on='patient_id'
)

# Aggiunta del timepoint baseline (mese 0)
baseline_for_merge = baseline_data.copy()
baseline_for_merge['visit_month'] = 0
baseline_for_merge['score'] = baseline_for_merge['baseline_score']
baseline_for_merge = baseline_for_merge[['patient_id', 'visit_month', 'score', 
                                        'age', 'gender', 'treatment', 'baseline_score']]

# Dataset completo con tutti i timepoint
full_data = pd.concat([baseline_for_merge, long_data], ignore_index=True)

print("Dataset longitudinale completo:")
print(full_data.head(10))

# Analisi dell'andamento temporale per gruppo di trattamento
temporal_means = full_data.groupby(['treatment', 'visit_month'])['score'].agg(['mean', 'std', 'count'])
print("\nAndamento temporale per gruppo di trattamento:")
print(temporal_means)

# Calcolo del cambiamento rispetto al baseline
# Pivot per avere i punteggi di ogni visita come colonne
patient_changes = full_data.pivot_table(
    index='patient_id', 
    columns='visit_month', 
    values='score'
)

# Rinomina le colonne per chiarezza
patient_changes = patient_changes.rename(columns={
    0: 'baseline', 
    3: 'month3', 
    6: 'month6', 
    12: 'month12'
})

# Calcola i cambiamenti
patient_changes['change_3m'] = patient_changes['month3'] - patient_changes['baseline']
patient_changes['change_6m'] = patient_changes['month6'] - patient_changes['baseline']
patient_changes['change_12m'] = patient_changes['month12'] - patient_changes['baseline']

# Reset dell'indice per unione
patient_changes = patient_changes.reset_index()

# Unione con caratteristiche baseline
patient_summary = pd.merge(
    patient_changes,
    baseline_data[['patient_id', 'treatment', 'age', 'gender']],
    on='patient_id'
)

print("\nCambiamenti rispetto al baseline per paziente:")
print(patient_summary[['patient_id', 'treatment', 'baseline', 'month12', 'change_12m']])

# Analisi per gruppo di trattamento
treatment_changes = patient_summary.groupby('treatment')[['change_3m', 'change_6m', 'change_12m']].mean()
print("\nCambiamento medio per gruppo di trattamento:")
print(treatment_changes)

Dataset longitudinale completo:
   patient_id  visit_month  score  age gender treatment  baseline_score
0           1            0     50   45      M         A              50
1           2            0     55   52      F         A              55
2           3            0     60   67      M         A              60
3           4            0     45   38      F         A              45
4           5            0     65   71      M         A              65
5           6            0     52   49      F         B              52
6           7            0     58   55      M         B              58
7           8            0     63   62      F         B              63
8           9            0     48   41      M         B              48
9          10            0     56   58      F         B              56

Andamento temporale per gruppo di trattamento:
                        mean       std  count
treatment visit_month                        
A         0            55.00  7.9056

Utilità:
- Ordinamento e calcolo di intervalli temporali
- Analisi per paziente o sottogruppo
- Calcolo della progressione (delta valori nel tempo)

{Grafico temporale con andamento di biomarcatori in un paziente nel tempo}

# Numpy

NumPy è una libreria Python che fornisce una struttura dati semplice ma potente: l'array n-dimensionale.

**Perché scegliere NumPy**
Pur conoscendo già Python “puro” (con i suoi cicli `for`, la lettura/scrittura di CSV, ecc.), NumPy introduce un paradigma che offre vantaggi concreti:

1. **Maggiore velocità**

   * NumPy utilizza algoritmi implementati in C, che eseguono operazioni in nanosecondi anziché in secondi.
2. **Riduzione dei cicli**

   * Grazie alle strutture array, è possibile comporre operazioni vettoriali che eliminano gran parte dei loop manuali e l’indice di iterazione.
3. **Codice più leggibile**

   * Senza cicli annidati, le espressioni nel codice assomigliano molto di più alle equazioni matematiche che si vogliono calcolare.
4. **Alta qualità e affidabilità**

   * Un’ampia comunità di sviluppatori mantiene NumPy veloce, di facile utilizzo e privo di bug.

**Conclusione**
Questi fattori hanno reso NumPy lo standard “de facto” per gli array multidimensionali in Python applicato alla data science. Molte librerie popolari si basano su NumPy: impararlo fornisce una solida base su cui poi sviluppare competenze più avanzate in ambiti specifici.


In [ ]:
%%bash
# attiviamo l'ambiente virutale
. .venv/bin/activate
# Installiamo numpy
pip install numpy

  Using cached numpy-2.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)


In [13]:
import numpy as np

CURVE_CENTER = 80 # Sintassi standard per le costanti
grades = np.array([72, 35, 64, 88, 51, 90, 74, 12])
grades_list = [72, 35, 64, 88, 51, 90, 74, 12]

print(grades)
print(grades_list)

[72 35 64 88 51 90 74 12]
[72, 35, 64, 88, 51, 90, 74, 12]


In [15]:
print(type(grades))
print(type(grades_list))

<class 'numpy.ndarray'>
<class 'list'>


In [14]:
def curve(grades):
     average = grades.mean()
     change = CURVE_CENTER - average
     new_grades = grades + change
     return np.clip(new_grades, grades, 100)

curve(grades)

array([ 91.25,  54.25,  83.25, 100.  ,  70.25, 100.  ,  93.25,  31.25])

1. **Importazione di NumPy**
   Alla riga 1 si importa la libreria NumPy con l’alias `np`, una convenzione che abbrevia i comandi successivi.

2. **Creazione dell’array**
   Alla riga 3 si definisce un array monodimensionale chiamato `grades` di lunghezza 8 e tipo `int64`. In seguito esplorerai forma e tipo dei dati più a fondo, ma per ora basti sapere che hai un vettore di 8 valori interi.

3. **Calcolo della media**
   Alla riga 5 si richiama il metodo `.mean()` sull’array: in un solo passaggio NumPy somma tutti gli elementi e ne restituisce la media. Gli array di NumPy dispongono di numerosi metodi analoghi per operazioni statistiche o matematiche.

4. **Vectorization e Broadcasting**
   Alla riga 7 si sfruttano due concetti chiave:

   * **Vectorization**: l’operazione (es. somma di uno stesso valore) viene applicata simultaneamente a tutti gli elementi dell’array, eliminando la necessità di cicli espliciti.
   * **Broadcasting**: NumPy “allinea” array di forme diverse per permettere calcoli vettoriali fra loro. Qui `grades` è un array shape `(8,)` e `change` è uno scalare shape `(1,)`; NumPy aggiunge automaticamente `change` a ciascun elemento di `grades`. Non avrebbe funzionato se grades fosse stata una lista!

5. **Clipping dei valori**
   Alla riga 8 si utilizza la funzione `np.clip()`, che garantisce che i voti “curve‑dati” non scendano sotto un minimo né superino un massimo.

   * Il secondo argomento di `clip()` è proprio l’array originale `grades`: così ogni voto corretto non scende mai al di sotto del suo valore iniziale.
   * Il terzo argomento è lo scalare `100`, che tramite broadcasting assicura che nessun voto ecceda il 100%.

> **Consiglio**: NumPy mette a disposizione decine di funzioni e metodi specializzati. Se ti sembra di ripetere operazioni comuni o di dover scrivere cicli, consulta la documentazione: molto probabilmente esiste già una routine adatta.


In [ ]:
%%bash
# installiamo matplotlib
pip install numpy matplotlib

## Forme degli array (Shape)

* **Shape** è la tupla che descrive la dimensione di ciascun asse di un array.
* Ogni array NumPy espone la proprietà `.shape`, che restituisce tale tupla.
* È fondamentale che gli array abbiano la forma che le funzioni si aspettano: un controllo rapido è stampare l’array insieme a `array.shape`.

**Esempio: creazione e verifica di un array 3D 2×2×3**

In [ ]:
import numpy as np

# Creo un vettore di 12 temperature e lo rimodello in un blocco 2×2×3
temperatures = np.array([
    29.3, 42.1, 18.8, 16.1, 38.0, 12.5,
    12.6, 49.9, 38.6, 31.3,  9.2, 22.2
]).reshape(2, 2, 3)
# Verifico la shape
print(temperatures.shape)

# Visualizzo l’array
print(temperatures)

(2, 2, 3)
[[[29.3 42.1 18.8]
  [16.1 38.  12.5]]

 [[12.6 49.9 38.6]
  [31.3  9.2 22.2]]]


In [ ]:
# Supponiamo di voler selezionare la temperatura 22.2
temperature = temperatures[1, 1, 2]
# ["tabella", "riga", "colonn"]
temperature

np.float64(22.2)

* Con tre o più dimensioni diventa difficile “vedere” i dati; per questo è utile

  * ribaltare (“swap”) assi con `.swapaxes()`
  * stampare shape e contenuto finché non si è certi della disposizione.

**Esempio: scambiare l’asse 1 con l’asse 2**

```python
reordered = np.swapaxes(temperatures, 1, 2)
print(reordered)
# Output:
# array([[[29.3, 16.1],
#         [42.1, 38. ],
#         [18.8, 12.5]],
#
#        [[12.6, 31.3],
#         [49.9,  9.2],
#         [38.6, 22.2]]])
```

---

## Assi (Axes)

* Gli **assi** indicano le dimensioni ed sono indicizzati da 0 in su.

  * In un array 2D, `axis=0` è l’asse verticale (righe), `axis=1` quello orizzontale (colonne).
* Molte funzioni NumPy cambiano comportamento a seconda dell’argomento `axis`.

**Esempio con `.max()`**

```python
import numpy as np

table = np.array([
    [5, 3, 7, 1],
    [2, 6, 7, 9],
    [1, 1, 1, 1],
    [4, 3, 2, 0],
])

# Massimo sull’intero array
print(table.max())        # 9

# Massimo per ciascuna colonna (asse 0)
print(table.max(axis=0))  # array([5, 6, 7, 9])

# Massimo per ciascuna riga (asse 1)
print(table.max(axis=1))  # array([7, 9, 1, 4])
```

* **Senza `axis`** → funzione su tutti i valori.
* **Con `axis=k`** → operazione lungo l’asse k, restituisce un array con dimensione ridotta di 1.

---

## Broadcasting

La regola fondamentale:

> Due array possono “broadcastarsi” se, per ogni asse, o hanno la stessa lunghezza, oppure almeno uno dei due è pari a 1.

* Se in un asse una dimensione vale 1, NumPy **duplica** quel dato lungo quell’asse.
* Se le dimensioni coincidono, operazione elemento-per-elemento.

**Esempio formale:**

* `A.shape = (4, 1, 8)`
* `B.shape = (1, 6, 8)`

| Asse   | A | B | Azione                 |
| ------ | - | - | ---------------------- |
| asse 0 | 4 | 1 | B duplicato 4 volte    |
| asse 1 | 1 | 6 | A duplicato 6 volte    |
| asse 2 | 8 | 8 | dimensioni uguali → OK |

**Creazione degli array**

```python
A = np.arange(32).reshape(4, 1, 8)
B = np.arange(48).reshape(1, 6, 8)
```

**Somma con broadcasting**

```python
C = A + B
print(C)
# Output:
# array([[[ 0,  2,  4,  6,  8, 10, 12, 14],
#         [ 8, 10, 12, 14, 16, 18, 20, 22],
#         [16, 18, 20, 22, 24, 26, 28, 30],
#         [24, 26, 28, 30, 32, 34, 36, 38],
#         [32, 34, 36, 38, 40, 42, 44, 46],
#         [40, 42, 44, 46, 48, 50, 52, 54]],
#
#        [[ 8, 10, 12, 14, 16, 18, 20, 22],
#         ...
#        ],
#        ...
# ])
```

* NumPy estende internamente A e B alle stesse dimensioni 4×6×8, poi somma elemento per elemento.

---

### Conclusione

* **Shape** e **axes** sono la base per navigare array multidimensionali.
* Controlla sempre `.shape` e, quando serve, usa `.swapaxes()` o funzioni con `axis=`.
* Comprendere il **broadcasting** permette di scrivere calcoli vettoriali puliti e veloci, evitando loop espliciti.
